In [1]:
%matplotlib qt
import matplotlib.pyplot as plt
import h5py, tifffile, os, numpy as np, multiprocessing as mp, gc, time
from functools import partial
from TXM_Sandbox.TXM_Sandbox.utils.xanes_math import (mrtv_reg1, mrtv_reg2, mrtv_reg3,
                                                      mrtv_mpc_combo_reg, tv_l1_pixel, 
                                                      tv_l1_subpixel, mrtv_ls_combo_reg)
from scipy.ndimage import fourier_shift

In [ ]:
fn = '/run/media/xiao/Data/data/Tiankai/66881_1080_mask_415p.tif'
obj_supp = tifffile.imread(fn).astype(np.int8)
fn = '/run/media/xiao/Data/data/Tiankai/66881_1080_mask_415p_sinogram.tif'
sino_supp = tifffile.imread(fn).astype(np.int8)

#supp = np.ones([415,415], dtype=np.int8)
fn = '/run/media/xiao/Data/data/Tiankai/66881_1080_mask_415p.h5'
with h5py.File(fn, 'w') as f:
    f.create_dataset('obj_supp', data=obj_supp, dtype=np.int8)
    f.create_dataset('sin_supp', data=sino_supp, dtype=np.int8)

In [ ]:
fn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_66883.h5'
ofn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_66883_roi.h5'

with h5py.File(fn, 'r') as f:
    img = ((f['img_tomo'][..., 1000:1440] - f['img_dark_avg'][..., 1000:1440])/
           (f['img_bkg_avg'][..., 1000:1440] - f['img_dark_avg'][..., 1000:1440]))
    img[np.isinf(img)] = 0
    img[np.isnan(img)] = 0
    

In [ ]:
print(img.shape)

In [ ]:
with h5py.File(ofn, 'w') as of:
    of.create_dataset('img_tomo', data=img.astype(np.float32), dtype=np.float32)

# ***Align projection image***

In [50]:
fn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_66797.h5'
ofn = '/run/media/xiao/Data/data/Tiankai/fly_scan2_id_6797.h5'
roi = [1000, 1600, 900, 1700]
missing_wedge = [121, 225]

with h5py.File(fn, 'r') as f:
    ang_len = f['angle'].shape[0]
ang = np.concatenate((np.arange(missing_wedge[0]), np.arange(missing_wedge[1], ang_len)))

In [45]:
print(ang_len)

386


In [28]:
with h5py.File(fn, 'r') as f:
    img = ((f['img_tomo'][ang, roi[0]:roi[1], roi[2]:roi[3]] - f['img_dark_avg'][:, roi[0]:roi[1], roi[2]:roi[3]])/
           (f['img_bkg_avg'][:, roi[0]:roi[1], roi[2]:roi[3]] - f['img_dark_avg'][:, roi[0]:roi[1], roi[2]:roi[3]]))
    img[np.isinf(img)] = 0
    img[np.isnan(img)] = 0

In [29]:
print(f"{id(img), img.nbytes, img.base}")

(140453016798752, 541440000, None)


In [30]:
#plt.plot(ang)
for ii in range(120, 123):
    plt.figure(ii)
    plt.imshow(img[ii])

###  ***align with mrtv_ls_combo_reg***

In [7]:
print(time.asctime())
n_cpu = os.cpu_count()
with mp.get_context('spawn').Pool(n_cpu-1) as pool:
    rlt1 = pool.map(partial(mrtv_ls_combo_reg, 100, 2, 10, 10, 0.2),
                    [[img[ii], img[ii+1]] for ii in range(img.shape[0]-1)])
print(time.asctime())

In [8]:
shift1 = []
for ii in range(len(rlt1)):
    shift1.append(rlt1[ii][-1])
shift1 = np.array(shift1)
#del(rlt1)
#gc.collect()

In [12]:
plt.figure(0)
plt.plot(shift1[:, 0])
plt.figure(1)
plt.plot(shift1[:, 1])


###  ***align with mrtv_reg3***

In [31]:
print(time.asctime())
n_cpu = os.cpu_count()
with mp.get_context('spawn').Pool(n_cpu-1) as pool:
    rlt2 = pool.map(partial(mrtv_reg3, 5, 10, 10, 0.2),
                   [[img[ii], img[ii+1]] for ii in range(img.shape[0]-1)])
print(time.asctime())

Thu Dec 10 00:04:44 2020
Thu Dec 10 00:11:23 2020


In [32]:
shift2 = []
for ii in range(len(rlt2)):
    shift2.append(rlt2[ii][-1])
shift2 = np.array(shift2)
shift2_cum = np.cumsum(shift2, axis=0)
#del(rlt2)
gc.collect()

13991

In [33]:
print(img.shape, shift2.shape, theta.shape, len(rlt2))

(282, 600, 800) (281, 2) (282,) 281


In [36]:
plt.figure(10)
plt.plot(shift2[:, 0])
plt.figure(11)
plt.plot(shift2[:, 1])

plt.figure(12)
plt.plot(shift2_cum[:, 0])
plt.figure(13)
plt.plot(shift2_cum[:, 1])

In [37]:
img_shifted = np.asarray(img)
img_shifted[0] = img[0]
for ii in range(shift2_cum.shape[0]):
    img_shifted[ii+1] = np.real(np.fft.ifftn(fourier_shift(np.fft.fftn(img[ii+1]), 
                                                           shift2_cum[ii])))

In [54]:
with h5py.File(fn, 'r') as f:
    theta = f['angle'][:]
    mag = f['Magnification'][()]
    pxl = f['Pixel Size'][()]
    eng = f['X_eng'][()]
    st = f['scan_time'][()]
with h5py.File(ofn, 'w') as f:
    f.create_dataset('img_tomo', data=np.concatenate((img_shifted[:missing_wedge[0]], np.zeros([ang_len-len(ang), *img_shifted.shape[1:]], dtype=np.float32), img_shifted[missing_wedge[0]:])).astype(np.float32), dtype=np.float32)
    f.create_dataset('img_bkg_avg', data=np.ones([1, *img.shape[1:]], dtype=np.float32), dtype=np.float32)
    f.create_dataset('img_dark_avg', data=np.zeros([1, *img.shape[1:]], dtype=np.float32), dtype=np.float32)
    f.create_dataset('img_bkg', data=np.ones([1, *img.shape[1:]], dtype=np.float32), dtype=np.float32)
    f.create_dataset('img_dark', data=np.zeros([1, *img.shape[1:]], dtype=np.float32), dtype=np.float32)
    f.create_dataset('angle', data=theta.astype(np.float32), dtype=np.float32)
    f.create_dataset('roi', data=np.array(roi), dtype=np.int32)
    f.create_dataset('missing_ang_range', data=np.array(missing_wedge), dtype=np.int32)
    f.create_dataset('align_alg', data=str('mrtv_reg3'))
    f.create_dataset('scan_time', data=st)
    f.create_dataset('X_eng', data=eng)
    f.create_dataset('Pixel Size', data=pxl)
    f.create_dataset('Magnification', data=mag)
    f.create_dataset('note', data=str('aligned with mrtv_reg3 with 5 levels and finest precision 0.2 pixel'))
    

In [44]:
print(theta.shape)

(282,)
